In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.stats as stats
import pickle
import obspy
from scipy.signal import find_peaks
import os
import pandas as pd

In [2]:
combined = pd.read_csv('/home/earthquakes1/homes/Rebecca/phd/stf/data/combined.csv')

In [3]:
window = 2

In [4]:
#scardec_name = os.listdir('/home/earthquakes1/homes/Rebecca/phd/stf/data/scardec')[0]

In [13]:
def look_at_maximums_in_windows(window_proportion = 0.1, window = None):
    number_of_peaks_opt = []
    magnitudes_opt = []
    number_of_peaks_moy = []
    magnitudes_moy = []
    names_of_peaks = []

    time_in_window_moy = []
    height_in_window_moy = []
    proportional_height_moy = []
    time_in_window_opt = []
    height_in_window_opt = []
    proportional_height_opt = []


    for scardec_name in os.listdir('/home/earthquakes1/homes/Rebecca/phd/stf/data/scardec'):
        names_of_peaks.append(scardec_name)
        db = combined[combined['scardec_name']==scardec_name]
        magnitudes_opt.append(db['scardec_magnitude'].values[0])
        magnitudes_moy.append(db['scardec_magnitude'].values[0])

        time_opt = []
        momentrate_opt = []

        time_moy = []
        momentrate_moy = []

        event = os.listdir(f'/home/earthquakes1/homes/Rebecca/phd/stf/data/scardec/{scardec_name}')
        starts = [n for n, l in enumerate(event) if l.startswith('fctopt')]
        with open(f'/home/earthquakes1/homes/Rebecca/phd/stf/data/scardec/{scardec_name}/{event[starts[0]]}') as f:
            lines = f.read().splitlines()

        lines = lines[2:]
        for line in lines:
            split = line.split(' ')
            split = [s for s in split if s not in ['', ' ', '\n']]
            time_opt.append(float(split[0]))
            momentrate_opt.append(float(split[1]))

        if window is None:
            window_opt = time_opt[-1] * window_proportion
        else:
            window_opt = window

        momentrate_window_opt = []

        for i, t in enumerate(time_opt):
            if t < window_opt:
                momentrate_window_opt.append(momentrate_opt[i])


        peaks_opt = find_peaks(momentrate_window_opt,
                        height = (None, None),
                        threshold = (None, None),
                        prominence = (None, None),
                        width = (None, None))

        starts = [n for n, l in enumerate(event) if l.startswith('fctmoy')]

        with open(f'/home/earthquakes1/homes/Rebecca/phd/stf/data/scardec/{scardec_name}/{event[starts[0]]}') as f:
            lines = f.read().splitlines()

        lines = lines[2:]
        #print(lines)
        for line in lines:
            split = line.split(' ')
            split = [s for s in split if s not in ['', ' ', '\n']]
            #print(split)
            time_moy.append(float(split[0]))
            momentrate_moy.append(float(split[1]))

        if window is None:
            window_moy = time_opt[-1] * window_proportion
        else:
            window_moy = window

        momentrate_window_moy = []
        for i, t in enumerate(time_moy):
            if t < window_moy:
                momentrate_window_moy.append(momentrate_moy[i])


        peaks_moy = find_peaks(momentrate_window_moy,
                        height = (None, None),
                        threshold = (None, None),
                        prominence = (None, None),
                        width = (None, None))

        try:
            max_peak_index = np.argmax(peaks_moy[1]['peak_heights'])
            max_height = peaks_moy[1]['peak_heights'][max_peak_index]
            max_time = time_moy[peaks_moy[0][max_peak_index]]
        except:
            max_peak_index = len(momentrate_window_moy)-1
            max_height = max(momentrate_window_moy)
            max_time = time_moy[max_peak_index]

        time_in_window_moy.append(max_time)
        height_in_window_moy.append(max_height)
        proportional_height_moy.append(max_height/np.max(momentrate_moy))

        try:
            max_peak_index = np.argmax(peaks_opt[1]['peak_heights'])
            max_height = peaks_opt[1]['peak_heights'][max_peak_index]
            max_time = time_moy[peaks_opt[0][max_peak_index]]
        except:
            max_peak_index = len(momentrate_window_opt)-1
            max_height = max(momentrate_window_opt)
            max_time = time_opt[max_peak_index]

        time_in_window_opt.append(max_time)
        height_in_window_opt.append(max_height)
        proportional_height_opt.append(max_height/np.max(momentrate_opt))

    return time_in_window_moy, height_in_window_moy, proportional_height_moy, time_in_window_opt, height_in_window_opt, proportional_height_opt, magnitudes_moy, magnitudes_opt



In [14]:
def plots_given_window_length(proportional_height_opt, proportional_height_moy, magnitudes_opt, magnitudes_moy, window):
    plt.scatter(magnitudes_opt, proportional_height_opt, label='opt', alpha = 0.5)
    plt.scatter(magnitudes_moy, proportional_height_moy, label='moy', alpha = 0.5)
    plt.ylabel(f'Proportional height of max in first {window} seconds')
    plt.xlabel('Magnitude')
    plt.legend()
    plt.savefig(f'/home/earthquakes1/homes/Rebecca/phd/stf/figures/peak_investigations/proportional_height_of_max_in_first_{window}s_vs_magnitude.png')
    plt.close()

    plt.hist(proportional_height_opt, bins = np.arange(0, 1.01, 0.05), alpha = 0.5, label='opt')
    plt.hist(proportional_height_moy, bins = np.arange(0, 1.01, 0.05), alpha = 0.5, label='moy')
    plt.xlabel(f'Proportional height of max in first {window} seconds')
    plt.ylabel('Number of events')
    plt.legend()
    plt.savefig(f'/home/earthquakes1/homes/Rebecca/phd/stf/figures/peak_investigations/proportional_height_of_max_in_first_{window}s_histogram.png')
    plt.close()

In [19]:
def plots_given_window_proportion(proportional_height_opt, proportional_height_moy, magnitudes_opt, magnitudes_moy, window_prop):
    plt.scatter(magnitudes_opt, proportional_height_opt, label='opt', alpha = 0.5)
    plt.scatter(magnitudes_moy, proportional_height_moy, label='moy', alpha = 0.5)
    plt.ylabel(f'Proportional height of max in first {window_prop:.1f} of earthquake')
    plt.xlabel('Magnitude')
    plt.legend()
    plt.savefig(f'/home/earthquakes1/homes/Rebecca/phd/stf/figures/peak_investigations/proportional_height_of_max_in_first_{window_prop:.1f}_of_eq_vs_magnitude.png')
    plt.close()

    plt.hist(proportional_height_opt, bins = np.arange(0, 1.01, 0.05), alpha = 0.5, label='opt')
    plt.hist(proportional_height_moy, bins = np.arange(0, 1.01, 0.05), alpha = 0.5, label='moy')
    plt.xlabel(f'Proportional height of max in first {window_prop:.1f} of earthquake')
    plt.ylabel('Number of events')
    plt.legend()
    plt.savefig(f'/home/earthquakes1/homes/Rebecca/phd/stf/figures/peak_investigations/proportional_height_of_max_in_first_{window_prop:.1f}_of_eq_histogram.png')
    plt.close()

In [20]:
for prop in np.arange(0.1, 1.01, 0.1):
    time_in_window_moy, height_in_window_moy, proportional_height_moy, time_in_window_opt, height_in_window_opt, proportional_height_opt, magnitudes_moy, magnitudes_opt = look_at_maximums_in_windows(window_proportion = prop)
    plots_given_window_proportion(proportional_height_opt, proportional_height_moy, magnitudes_opt, magnitudes_moy, prop)